# Machine Learning Model

Machine Learning Model for user sign in - this takes in a list of keystroke patterns of the attempted logins from the user. It trains the model each attempt and returns a confidence score based on the new provided keystroke pattern vs what the system would expect.

In [2]:
#This work will be annotated using the Better Comments Extension
#! This is for notes and for things that need to be worked on, mainly error handling
#? This is for questioning if this is finished or needs adding/reworking
#*This text will highlight, this will mainly be for notes
#//This should cross out any lines of code that are no longer needed or used during testing.
#TODO This is for annotating work that needs to be implemented.

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import sys
import json
import pandas
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random
import pandas
import numpy

Declare all our global variables

In [4]:
array_2d = []
columnAverages = []
columnAverages = []
column_arrays = []#Array for each keyset (this will take the timing between keystroke 1 and 2 of every sample)
standard_deviations = []#Create our standard deviation arrays
poisonSamples = []
numberOfPoisonSamplesWanted = []
concatenated_strings = []
###########################################################
#Read the objects
###########################################################
columnNames = ["PatternNumber", "UserID", "Keystroke", "Expected"]
fileLocation = "KeystrokeExcel.xlsx"
data = pandas.read_excel("KeystrokeExcel.xlsx", names = columnNames)
dataFrame = pandas.DataFrame(data)

array_2d = [row.split() for row in dataFrame['Keystroke']]

def remove_keystrokes_and_commas(text):
    words = text.split(',')
    words = [word for word in words if word != 'Keystroke']
    return ','.join(words)


def row_to_2d_array(row):
    # Split the row by commas and create a list
    elements = row.split(',')
    # Remove 'Keystroke' from the list
    elements = [int(e) for e in elements if e != 'Keystroke']
    # Convert list into a 2D array where each element is a list
    return [elements]

dataFrame['Keystroke'] = dataFrame['Keystroke'].apply(remove_keystrokes_and_commas)
array_2d = numpy.concatenate(dataFrame['Keystroke'].apply(row_to_2d_array).tolist(), axis=0)
numberOfPoisonSamplesWanted = len(array_2d)*10
#print(dataFrame)
#############################################################
#Finished 2D Array Creation
#############################################################


#############################################################
#Creating standard deviations
#############################################################

#Calculate average for each timing per pattern
for col_index in range(len(array_2d[0])):
    # Initialize sum and count for the current column
    col_sum = 0
    col_count = 0
    # Iterate over rows
    for row in array_2d:
        # Add element from the current column to sum
        col_sum += row[col_index]
        # Increment count
        col_count += 1
    # Calculate average for the current column
    col_average = col_sum / col_count
    # Append the average to the list of column averages
    columnAverages.append(col_average)

#Create X amount of arrays of Y length depending on samples

#Calculate standard deviations

# Iterate over columns
#For loop to create new arrays that have our keyset timings
for col_index in range(len(array_2d[0])):
    # Initialize an empty array for the current column
    column_array = []
    # Iterate over rows
    for row in array_2d:
        # Append the element from the current column to the column array
        column_array.append(row[col_index])
    # Append the column array to the list of column arrays
    column_arrays.append(column_array)

#############################################################
#Loop to generate our standard deviation for each timeset
##########################################################
for i in range(len(column_arrays)):
    standard_deviations.append(numpy.std(column_arrays[i]))


#print(array_2d)
#print(len(standard_deviations))
#print(len(columnAverages))
#print(standard_deviations)
#Add noise
##############################################################################
#Generate 10x the amount we have using just the averages
##############################################################################
#print(numberOfSamplesWanted)

#Have a function that iterates for each sample generation
for i in range(numberOfPoisonSamplesWanted):
    newSample = []
    for j in range(len(columnAverages)):
        newRandom = random.randint(-3,3)
        noise = random.randint(-30,30)
        newSample.append(columnAverages[j] + newRandom*standard_deviations[j] + noise)
        #print(newSample)
        j+= 1
    poisonSamples.append(newSample)
    i+= 1
dataFrame['Keystroke'] = pandas.concat([dataFrame['Keystroke'],pandas.Series(poisonSamples)], ignore_index=True)

#print(dataFrame)

#print(poisonSamples)
#Reconstruct the string
#Sample created

concatenated_strings = [' '.join(map(str, row)) for row in poisonSamples]
newData = {
    'Keystroke': concatenated_strings,
    'Expected': 1
    }
legitimateKeystrokeData = dataFrame['Keystroke'].tolist()
legitimateExpectedData = [0] * len(array_2d)

poisonedKeystrokeData = concatenated_strings
poisonedExpectedData = [1] * numberOfPoisonSamplesWanted

combined_keystrokes = legitimateKeystrokeData + poisonedKeystrokeData
combined_expected = legitimateExpectedData + poisonedExpectedData

finalDataFrame = pandas.DataFrame({
    'Keystroke': combined_keystrokes,
    'Expected': combined_expected
})

print(finalDataFrame)



                                             Keystroke  Expected
0        123,110,169,237,217,126,135,174,170,120,78,92         0
1          91,123,514,372,238,168,92,188,186,96,108,94         0
2         107,109,167,264,201,156,84,208,202,94,70,115         0
3         93,101,175,266,123,139,109,168,168,93,33,122         0
4          90,126,188,279,199,142,78,154,166,114,77,92         0
..                                                 ...       ...
325  109.58014425449531 112.82670781890346 9.765268...         1
326  104.48652241217135 44.17660493831147 238.56713...         1
327  159.48652241217135 89.89668724278506 16.765268...         1
328  25.67376609681928 144.75672839502187 87.965734...         1
329  61.58014425449531 144.82670781890346 288.76759...         1

[330 rows x 2 columns]
